[View in Colaboratory](https://colab.research.google.com/github/zaojack/tensorflow/blob/master/tensor1.ipynb)

In [8]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)
sess=tf.InteractiveSession()
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x=tf.placeholder(tf.float32,[None,784])
y_=tf.placeholder(tf.float32,[None,10])

w_conv1=weight_variable([5,5,1,32])
b_conv1=bias_variable([32])

x_image=tf.reshape(x,[-1,28,28,1])

h_conv1=tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)
h_pool1=max_pool_2x2(h_conv1)

w_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])

h_conv2=tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)

w_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])

h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)

keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

w_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2)+b_fc2)
#y_conv=tf.nn.relu(tf.matmul(h_fc1_drop,w_fc2)+b_fc2)
#用relu 对于刚开始的时候获得不到梯度

#对于分类算法常用softmax!!!

cross_entropy=-tf.reduce_sum(y_*tf.log(y_conv))

train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


init=tf.global_variables_initializer()
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
# turn correct_prediction to float32

sess.run(init)
for i in range(10000):
  batch = mnist.train.next_batch(50)
  if i%100==0:
    train_accuracy=accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.})
    print "step %d ,training_accuracy %g"%(i,train_accuracy)
  train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})   
print "test_accuracy %g"%accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0})    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0 ,training_accuracy 0.1
step 100 ,training_accuracy 0.86
step 200 ,training_accuracy 0.9


KeyboardInterrupt: ignored